In [1]:
from remote_run import remote_run, to_string, viz
from jetlag import Universal, RemoteJobWatcher
from knownsystems import *
import sys
from random import randint

uv = Universal()
uv.load(
    backend=backend_tapis,
    notify='sbrandt@cct.lsu.edu',
    jetlag_id='rostam-sbrandt',
)

print("The complete list of valid jetlag_id's that can be used for this test:")
for sys in uv.systems():
    print(" ",sys)
print()

reading TAPIS_USER from /home/jovyan/.TAPIS_USER...
Token refresh successful
The complete list of valid jetlag_id's that can be used for this test:
  rostam-sbrandt
  shelob-funwave



In [2]:
def fib(n):
    if n < 2:
        return n
    else:
        return fib(n-1)+fib(n-2)

fibno = randint(13,20)
print('fib(',fibno,')=...',sep='',flush=True)

job = remote_run(uv, fib, (fibno,), nodes=1, ppn=1)
job.wait()
print("result:",job.get_result())

try:
    viz(job)
except:
    print("Exception during viz step:",sys.exc_info()[0])

fib(20)=...
Job ID: 2464a31d-e4a7-4ed7-a6ba-7389060f9dd0-007
STAGING_INPUTS
SUBMITTING
RUNNING
Cleanup: rostam-sbrandt-storage-tg457049/tjob/tg457049/py-fun-1137592609...done
FINISHED
tar: run_dir/OTF2_archive/APEX/0.def: time stamp 2020-03-23 19:18:47 is 283339.169043281 s in the future
tar: run_dir/OTF2_archive/APEX/1.def: time stamp 2020-03-23 19:18:47 is 283339.168541589 s in the future
tar: run_dir/OTF2_archive/APEX/2.def: time stamp 2020-03-23 19:18:47 is 283339.168288089 s in the future
tar: run_dir/OTF2_archive/APEX/3.def: time stamp 2020-03-23 19:18:47 is 283339.167957239 s in the future
tar: run_dir/OTF2_archive/APEX/4.def: time stamp 2020-03-23 19:18:47 is 283339.167619022 s in the future
tar: run_dir/OTF2_archive/APEX/5.def: time stamp 2020-03-23 19:18:47 is 283339.165116847 s in the future
tar: run_dir/OTF2_archive/APEX/6.def: time stamp 2020-03-23 19:18:47 is 283339.164539647 s in the future
tar: run_dir/OTF2_archive/APEX/7.def: time stamp 2020-03-23 19:18:47 is 283339.16

In [3]:
# You can look at data from an old job if you supply the full job id
#job = RemoteJobWatcher(uv, "2327d1e6-7efc-48d8-9807-dad8a9255b71-007")
import os
print("result:",job.get_result())
os.environ["PATH"]="/usr/local/otf2/bin:/home/jovyan/workenv/bin:/home/jovyan/bin:/usr/local/cli/bin:/traveler-integrated/notebook/agave-cli/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin"
try:
    viz(job)
except:
    print("Exception during viz step:",sys.exc_info()[0])

result: 6765


In [4]:
%%html
<iframe src="http://localhost:8000" style='width: 100%; height: 300px;'></iframe>

In [34]:
# This was an experiment in which @Phylanx was run locally.
import os
from visualizeInTraveler import visualizeRemoteInTraveler
pid = str(os.getpid())
def write_perf(f,label):
    t = f.__perfdata__
    s = f.get_python_src(f.backend.wrapped_function)
    ps = f.get_physl_source()
    dir="jobdata-"+pid+"/run_dir"
    os.makedirs(dir,exist_ok=True)
    with open(dir+"/py-csv.txt","w") as fd:
        print(t[0],end='',file=fd)
    with open(dir+"/py-tree.txt","w") as fd:
        print(t[1],end='',file=fd)
    with open(dir+"/py-graph.txt","w") as fd:
        print(t[2],end='',file=fd)
    with open(dir+"/py-src.txt","w") as fd:
        print(s,file=fd)
    with open(dir+"/physl-src.txt","w") as fd:
        print(ps,file=fd)
    with open(dir+"/label.txt","w") as fd:
        print(label,file=fd)
write_perf(f4,"f4(2)")
visualizeRemoteInTraveler(pid)

<Response [200]>